In [ ]:
# Importing all libraries.
from netCDF4 import Dataset
import os
import matplotlib.pyplot as plt, mpld3
import numpy as np
import math
from datetime import timedelta
from operator import attrgetter
import warnings
warnings.filterwarnings('ignore')
lon1 = -25
lon2 = -5
lat1 = 60
lat2 = 64

from mpl_toolkits.basemap import Basemap
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile,ErrorCode
#distance on a sphere
def compute_dist_sphere(xwant,ywant):
    dist=np.zeros_like(xwant)
    for ipt in range(1,len(xwant),1):
        # Approximate radius of earth in km
        R = 6373.0

        lat1 = np.deg2rad(ywant[ipt-1])
        lon1 = np.deg2rad(xwant[ipt-1])
        lat2 = np.deg2rad(ywant[ipt])
        lon2 = np.deg2rad(xwant[ipt])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

        dist[ipt] = R * c
        
    return np.cumsum(dist)

In [ ]:
#experiment='line_shelf_continuous'
#experiment='line_FBC_continuous'
#experiment='line_IFR_continuous'
if experiment=='line_shelf_continuous':
    
    nparts=20
    #the number of days to run the experiment
    T_exec=364
    
    DT_launch=1 #number of days between launch

    
    xsecs=[-16.3,-15.8]
    ysecs=[63.4,62.7]
    lon_init=np.linspace(xsecs[0],xsecs[1],nparts)
    lat_init=np.linspace(ysecs[0],ysecs[1],nparts)


    
if experiment=='line_FBC_continuous':
    
    nparts=10
    #the number of days to run the experiment
    T_exec=364
    T_stop_launch=300 #because on average they take 2 months to travel so no need to launch in the last 2 months
    
    DT_launch=1 #number of days between launch

    
    xsecs=[-9.5,-9.2]
    ysecs=[61.6,62]
    lon_init=np.linspace(xsecs[0],xsecs[1],nparts)
    lat_init=np.linspace(ysecs[0],ysecs[1],nparts)
    
if experiment=='line_IFR_continuous':
    
    nparts=30
    #the number of days to run the experiment
    T_exec=364
    T_stop_launch=300 #because on average they take 2 months to travel so no need to launch in the last 2 months
    
    DT_launch=1 #number of days between launch

    
    xsecs=[-11,-8.1]
    ysecs=[63.5,62.5]
    lon_init=np.linspace(xsecs[0],xsecs[1],nparts)
    lat_init=np.linspace(ysecs[0],ysecs[1],nparts)    

#be sure that it's in the u,v field ! 
dll=0.02 #the grid spacing in the interpolation
is_part_ok=np.full(lon_init.shape,True)
try :
    for ipart in range(len(lon_init)):
        take_tmp=(lon<(lon_init[ipart]+dll))&(lon>(lon_init[ipart]-dll))&(lat<(lat_init[ipart]+dll))&(lat>(lat_init[ipart]-dll))
        if np.isnan(vort.data[take_tmp][0]): is_part_ok[ipart]=False
except :
    pass
  


lon_init=lon_init[is_part_ok]
lat_init=lat_init[is_part_ok]



print('we initialize {} particles'.format(len(lon_init)))

#### run the advection

In [ ]:
path_save_lag='./output_lagrangian_'+experiment+'/'
try:
    os.mkdir(path_save_lag)
except:
    pass

namesave_lag=path_save_lag+'output_lagrangian_1year_2005.zarr'


## load the variables
filenames = {'U': filepath+namefile,
             'V': filepath+namefile}
variables = {'U': 'u',
             'V': 'v'}

dimensions = {'time': 'day','lat': 'latitude', 'lon': 'longitude'}



fieldset = FieldSet.from_netcdf(filenames, variables, dimensions,allow_time_extrapolation=True)


https://docs.oceanparcels.org/en/latest/examples/tutorial_delaystart.html#Using-the-repeatdt-argument

In [ ]:
#to delete particles when they leave the domain
def DeleteParticle(particle, fieldset, time):
    particle.delete()


if experiment=='line_shelf_continuous': 
    #launch particles every days

    repeatdt = timedelta(days=DT_launch)  # release from the same set of locations every ??

    pset = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                                 lon=lon_init,
                                 lat=lat_init,# give lists of position
                                 time=0*24*3600,#the time of beggining in seconds
                                 repeatdt=repeatdt)  
    
    output_file = pset.ParticleFile(name=namesave_lag, outputdt=timedelta(hours=6))
    pset.show(field=fieldset.U) #this show the current state of the particles : either at begginin or at the end



    pset.execute(AdvectionRK4,
             runtime=timedelta(days=T_exec),
             dt=timedelta(minutes=5),
             output_file=output_file,
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    
    
elif experiment=='line_FBC_continuous': 
    
    #launch particles every days then stop after a given time

    repeatdt = timedelta(days=DT_launch)  # release from the same set of locations every ??

    pset = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                                 lon=lon_init,
                                 lat=lat_init,# give lists of position
                                 time=0*24*3600,#the time of beggining in seconds
                                 repeatdt=repeatdt)     
    
    output_file = pset.ParticleFile(name=namesave_lag, outputdt=timedelta(hours=6))
    pset.show(field=fieldset.U) #this show the current state of the particles : either at begginin or at the end


    pset.execute(AdvectionRK4,
             runtime=timedelta(days=T_stop_launch),
             dt=timedelta(minutes=5),
             output_file=output_file,
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    
    
    # now stop the repeated release
    pset.repeatdt = None
    
    pset.execute(AdvectionRK4,
             runtime=timedelta(days=(T_exec-T_stop_launch)),
             dt=timedelta(minutes=5),
             output_file=output_file,
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    
elif experiment=='line_IFR_continuous': 
    
    #launch particles every days then stop after a given time

    repeatdt = timedelta(days=DT_launch)  # release from the same set of locations every ??

    pset = ParticleSet.from_list(fieldset=fieldset, pclass=JITParticle,
                                 lon=lon_init,
                                 lat=lat_init,# give lists of position
                                 time=0*24*3600,#the time of beggining in seconds
                                 repeatdt=repeatdt)     
    
    output_file = pset.ParticleFile(name=namesave_lag, outputdt=timedelta(hours=6))
    pset.show(field=fieldset.U) #this show the current state of the particles : either at begginin or at the end


    pset.execute(AdvectionRK4,
             runtime=timedelta(days=T_stop_launch),
             dt=timedelta(minutes=5),
             output_file=output_file,
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    
    
    # now stop the repeated release
    pset.repeatdt = None
    
    pset.execute(AdvectionRK4,
             runtime=timedelta(days=(T_exec-T_stop_launch)),
             dt=timedelta(minutes=5),
             output_file=output_file,
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    
